# <center> NoteBook : Enrichissement <center>

Afin de compléter notre analyse, nous décidons de procéder à divers enrichissements : 

<u> **PARTIE 1** </u> : 

- Association des Stations avec les Arrondissements
- Quartiers de Paris
- Gares dans Paris
    - Association des Arrondissements & Communes avec les Gares dans Paris
- Gares & futures gares dans le Grand Paris
    - Association des Arrondissements & Communes avec les Gares du Grand Paris
    
<u>**PARTIE 2**</u> :

- Fonction du calcul du nombre de stations autour d'une gare

In [1]:
import pandas as pd
import os
import geopandas as gpd
import geopy.distance


from shapely.geometry import Point

# ASSOCIATION STATION - ARRONDISSEMENT

In [2]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install geopy

In [3]:
stations = pd.read_csv('liste_stations_cleaned.csv')
stations.head()

,Identifiant station,Nom station,Station en fonctionnement,CapacitÃ© de la station,Nombre bornettes libres,Nombre total vÃ©los disponibles,VÃ©los mÃ©caniques disponibles,VÃ©los Ã©lectriques disponibles,Borne de paiement disponible,Retour vÃ©lib possible,Actualisation de la donnÃ©e,CoordonnÃ©es gÃ©ographiques,Nom communes Ã©quipÃ©es,Code INSEE communes Ã©quipÃ©es,station_velib_plus,geometry,Latitude,Longitude,arrondissement,quartier
0,44015,Rouget de L'isle - Watteau,NON,0,0,0,0,0,NON,NON,2023-02-09T10:35:37+01:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,NaN,False,NaN,48.778193,2.396302,Banlieue,Banlieue
1,16107,Benjamin Godard - Victor Hugo,OUI,35,32,3,1,2,OUI,OUI,2023-06-05T08:29:35+02:00,"48.865983, 2.275725",Paris,NaN,False,NaN,48.865983,2.275725,16ème Ardt,Porte-Dauphine
2,8026,Messine - Place Du PÃ©rou,OUI,12,1,11,5,6,OUI,OUI,2023-06-05T08:24:54+02:00,"48.875448033960744, 2.315508019010038",Paris,NaN,False,NaN,48.875448,2.315508,8ème Ardt,Europe
3,31104,Mairie de Rosny-sous-Bois,OUI,30,21,9,3,6,OUI,OUI,2023-06-05T08:28:44+02:00,"48.871256519012, 2.4865807592869",Rosny-sous-Bois,NaN,False,NaN,48.871257,2.486581,Banlieue,Banlieue
4,6003,Saint-Sulpice,OUI,21,20,1,1,0,OUI,OUI,2023-06-05T08:26:33+02:00,"48.85165383178419, 2.3308077827095985",Paris,NaN,False,NaN,48.851654,2.330808,6ème Ardt,Notre-Dame-des-Champs


In [4]:
stations[['Latitude', 'Longitude']] = stations['CoordonnÃ©es gÃ©ographiques'].str.split(',', 1, expand=True)
stations.head()

,Identifiant station,Nom station,Station en fonctionnement,CapacitÃ© de la station,Nombre bornettes libres,Nombre total vÃ©los disponibles,VÃ©los mÃ©caniques disponibles,VÃ©los Ã©lectriques disponibles,Borne de paiement disponible,Retour vÃ©lib possible,Actualisation de la donnÃ©e,CoordonnÃ©es gÃ©ographiques,Nom communes Ã©quipÃ©es,Code INSEE communes Ã©quipÃ©es,station_velib_plus,geometry,Latitude,Longitude,arrondissement,quartier
0,44015,Rouget de L'isle - Watteau,NON,0,0,0,0,0,NON,NON,2023-02-09T10:35:37+01:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,NaN,False,NaN,48.778192750803,2.3963020229163,Banlieue,Banlieue
1,16107,Benjamin Godard - Victor Hugo,OUI,35,32,3,1,2,OUI,OUI,2023-06-05T08:29:35+02:00,"48.865983, 2.275725",Paris,NaN,False,NaN,48.865983,2.275725,16ème Ardt,Porte-Dauphine
2,8026,Messine - Place Du PÃ©rou,OUI,12,1,11,5,6,OUI,OUI,2023-06-05T08:24:54+02:00,"48.875448033960744, 2.315508019010038",Paris,NaN,False,NaN,48.875448033960744,2.315508019010038,8ème Ardt,Europe
3,31104,Mairie de Rosny-sous-Bois,OUI,30,21,9,3,6,OUI,OUI,2023-06-05T08:28:44+02:00,"48.871256519012, 2.4865807592869",Rosny-sous-Bois,NaN,False,NaN,48.871256519012,2.4865807592869,Banlieue,Banlieue
4,6003,Saint-Sulpice,OUI,21,20,1,1,0,OUI,OUI,2023-06-05T08:26:33+02:00,"48.85165383178419, 2.3308077827095985",Paris,NaN,False,NaN,48.85165383178419,2.3308077827095985,6ème Ardt,Notre-Dame-des-Champs


In [5]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1464 entries, 0 to 1463
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Identifiant station              1464 non-null   int64  
 1   Nom station                      1464 non-null   object 
 2   Station en fonctionnement        1464 non-null   object 
 3   CapacitÃ© de la station          1464 non-null   int64  
 4   Nombre bornettes libres          1464 non-null   int64  
 5   Nombre total vÃ©los disponibles  1464 non-null   int64  
 6   VÃ©los mÃ©caniques disponibles   1464 non-null   int64  
 7   VÃ©los Ã©lectriques disponibles  1464 non-null   int64  
 8   Borne de paiement disponible     1464 non-null   object 
 9   Retour vÃ©lib possible           1464 non-null   object 
 10  Actualisation de la donnÃ©e      1464 non-null   object 
 11  CoordonnÃ©es gÃ©ographiques      1464 non-null   object 
 12  Nom communes Ã©quipÃ

In [6]:
# Changement de "Type"
stations['Latitude']  = stations['Latitude'].astype(float)
stations['Longitude'] = stations['Longitude'].astype(float)
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1464 entries, 0 to 1463
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Identifiant station              1464 non-null   int64  
 1   Nom station                      1464 non-null   object 
 2   Station en fonctionnement        1464 non-null   object 
 3   CapacitÃ© de la station          1464 non-null   int64  
 4   Nombre bornettes libres          1464 non-null   int64  
 5   Nombre total vÃ©los disponibles  1464 non-null   int64  
 6   VÃ©los mÃ©caniques disponibles   1464 non-null   int64  
 7   VÃ©los Ã©lectriques disponibles  1464 non-null   int64  
 8   Borne de paiement disponible     1464 non-null   object 
 9   Retour vÃ©lib possible           1464 non-null   object 
 10  Actualisation de la donnÃ©e      1464 non-null   object 
 11  CoordonnÃ©es gÃ©ographiques      1464 non-null   object 
 12  Nom communes Ã©quipÃ

In [7]:
arrondissements_station = []

# Charger les données des arrondissements
arrondissements = gpd.read_file('arrondissements.geojson')

# Créer une fonction pour trouver l'arrondissement d'une station
def trouver_arrondissement(latitude, longitude):
    
    # Créer un point à partir des coordonnées de la station
    point = Point(longitude, latitude)

    # Rechercher l'arrondissement correspondant au point
    arrondissement_serie = arrondissements[arrondissements.contains(point)]['l_ar']
    
    if not arrondissement_serie.empty:
        arrondissement = arrondissement_serie.iloc[0]
        return arrondissement
    else:
        return "Banlieue"

# Itérer sur chaque station
for index, station in stations.iterrows():
    latitude_station  = station['Latitude']
    longitude_station = station['Longitude']

    # Appeler la fonction pour trouver l'arrondissement de la station
    arrondissement_station = trouver_arrondissement(latitude_station, longitude_station)
    
    # Ajouter l'arrondissement à la liste
    arrondissements_station.append(arrondissement_station)

    # Ajouter la liste des arrondissements comme une colonne "arrondissement" dans le DataFrame des stations
stations['arrondissement'] = arrondissements_station

stations.head()

,Identifiant station,Nom station,Station en fonctionnement,CapacitÃ© de la station,Nombre bornettes libres,Nombre total vÃ©los disponibles,VÃ©los mÃ©caniques disponibles,VÃ©los Ã©lectriques disponibles,Borne de paiement disponible,Retour vÃ©lib possible,Actualisation de la donnÃ©e,CoordonnÃ©es gÃ©ographiques,Nom communes Ã©quipÃ©es,Code INSEE communes Ã©quipÃ©es,station_velib_plus,geometry,Latitude,Longitude,arrondissement,quartier
0,44015,Rouget de L'isle - Watteau,NON,0,0,0,0,0,NON,NON,2023-02-09T10:35:37+01:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,NaN,False,NaN,48.778193,2.396302,Banlieue,Banlieue
1,16107,Benjamin Godard - Victor Hugo,OUI,35,32,3,1,2,OUI,OUI,2023-06-05T08:29:35+02:00,"48.865983, 2.275725",Paris,NaN,False,NaN,48.865983,2.275725,16ème Ardt,Porte-Dauphine
2,8026,Messine - Place Du PÃ©rou,OUI,12,1,11,5,6,OUI,OUI,2023-06-05T08:24:54+02:00,"48.875448033960744, 2.315508019010038",Paris,NaN,False,NaN,48.875448,2.315508,8ème Ardt,Europe
3,31104,Mairie de Rosny-sous-Bois,OUI,30,21,9,3,6,OUI,OUI,2023-06-05T08:28:44+02:00,"48.871256519012, 2.4865807592869",Rosny-sous-Bois,NaN,False,NaN,48.871257,2.486581,Banlieue,Banlieue
4,6003,Saint-Sulpice,OUI,21,20,1,1,0,OUI,OUI,2023-06-05T08:26:33+02:00,"48.85165383178419, 2.3308077827095985",Paris,NaN,False,NaN,48.851654,2.330808,6ème Ardt,Notre-Dame-des-Champs


# ENRICHISSEMENT QUARTIER DE PARIS

In [8]:
quartiers_station = []

# Charger les données des quartiers
quartiers = gpd.read_file('quartier_paris.geojson')

# Créer une fonction pour trouver le quartier d'une station
def trouver_quartier(latitude, longitude):
    
    # Créer un point à partir des coordonnées de la station
    point = Point(longitude, latitude)

    # Rechercher le quartier correspondant au point
    quartier_serie = quartiers[quartiers.contains(point)]['l_qu']
    
    if not quartier_serie.empty:
        quartier   = quartier_serie.iloc[0]
        return quartier
    else:
        return "Banlieue"

# Itérer sur chaque station
for index, station in stations.iterrows():
    latitude_station  = station['Latitude']
    longitude_station = station['Longitude']

    # Appeler la fonction pour trouver le quartier de la station
    quartier_station  = trouver_quartier(latitude_station, longitude_station)
    
    # Ajouter le quartier à la liste
    quartiers_station.append(quartier_station)

    # Ajouter la liste des quartiers comme une colonne "quartier" dans le DataFrame des stations
stations['quartier']  = quartiers_station

stations.head()

,Identifiant station,Nom station,Station en fonctionnement,CapacitÃ© de la station,Nombre bornettes libres,Nombre total vÃ©los disponibles,VÃ©los mÃ©caniques disponibles,VÃ©los Ã©lectriques disponibles,Borne de paiement disponible,Retour vÃ©lib possible,Actualisation de la donnÃ©e,CoordonnÃ©es gÃ©ographiques,Nom communes Ã©quipÃ©es,Code INSEE communes Ã©quipÃ©es,station_velib_plus,geometry,Latitude,Longitude,arrondissement,quartier
0,44015,Rouget de L'isle - Watteau,NON,0,0,0,0,0,NON,NON,2023-02-09T10:35:37+01:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,NaN,False,NaN,48.778193,2.396302,Banlieue,Banlieue
1,16107,Benjamin Godard - Victor Hugo,OUI,35,32,3,1,2,OUI,OUI,2023-06-05T08:29:35+02:00,"48.865983, 2.275725",Paris,NaN,False,NaN,48.865983,2.275725,16ème Ardt,Porte-Dauphine
2,8026,Messine - Place Du PÃ©rou,OUI,12,1,11,5,6,OUI,OUI,2023-06-05T08:24:54+02:00,"48.875448033960744, 2.315508019010038",Paris,NaN,False,NaN,48.875448,2.315508,8ème Ardt,Europe
3,31104,Mairie de Rosny-sous-Bois,OUI,30,21,9,3,6,OUI,OUI,2023-06-05T08:28:44+02:00,"48.871256519012, 2.4865807592869",Rosny-sous-Bois,NaN,False,NaN,48.871257,2.486581,Banlieue,Banlieue
4,6003,Saint-Sulpice,OUI,21,20,1,1,0,OUI,OUI,2023-06-05T08:26:33+02:00,"48.85165383178419, 2.3308077827095985",Paris,NaN,False,NaN,48.851654,2.330808,6ème Ardt,Notre-Dame-des-Champs


# ENRICHISSEMENT LISTE DES GARES

In [10]:
# Charger les données de la liste des gares
gares = pd.read_csv("emplacement-des-gares-idf_1.csv", sep =';')
gares.head()

,Geo Point,Geo Shape,gares_id,nom_long,nom_so_gar,nom_su_gar,id_ref_ZdC,nom_ZdC,id_ref_ZdA,nom_ZdA,...,termetro,tertram,terval,exploitant,idf,principal,x,y,picto ligne,nom_iv
0,"48.68101661540746, 2.533130656571037","{""coordinates"": [2.533130656571037, 48.6810166...",105,Boussy-Saint-Antoine,NaN,NaN,62596,Boussy-Saint-Antoine,47924,Boussy-Saint-Antoine,...,0,0,0,SNCF,1,0,665628.8216,6.842419e+06,https://data.iledefrance-mobilites.fr/api/expl...,Boussy-Saint-Antoine
1,"48.60674494801255, 2.3020596915826386","{""coordinates"": [2.302059691582639, 48.6067449...",108,Brétigny,NaN,NaN,60247,Brétigny,43098,Brétigny,...,0,0,0,SNCF,1,1,648544.6367,6.834288e+06,https://data.iledefrance-mobilites.fr/api/expl...,Brétigny
2,"49.00975645120457, 2.5612326952251054","{""coordinates"": [2.561232695225105, 49.0097564...",743,Aéroport Charles De Gaulle 1,NaN,NaN,73596,Aéroport CDG 1 (Terminal 3),462398,Aéroport CDG 1 (Terminal 3) - RER,...,0,0,0,SNCF,1,0,667900.8037,6.878961e+06,https://data.iledefrance-mobilites.fr/api/expl...,Aéroport Charles De Gaulle 1
3,"48.798892499145914, 2.3281369230647506","{""coordinates"": [2.328136923064751, 48.7988924...",22,Arcueil-Cachan,NaN,NaN,70263,Arcueil - Cachan,43067,Arcueil - Cachan,...,0,0,0,RATP,1,0,650648.8903,6.855634e+06,https://data.iledefrance-mobilites.fr/api/expl...,Arcueil-Cachan
4,"48.932196005027805, 2.495513142584313","{""coordinates"": [2.495513142584313, 48.9321960...",956,Aulnay-sous-Bois,NaN,NaN,72646,Aulnay-sous-Bois,43071,Aulnay-sous-Bois,...,0,0,0,SNCF,1,1,663037.8693,6.870365e+06,https://data.iledefrance-mobilites.fr/api/expl...,Aulnay-sous-Bois
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,"48.795138178428715, 2.1351562241000215","{""coordinates"": [2.135156224100021, 48.7951381...",886,Versailles-Chantiers,NaN,NaN,63880,Versailles Chantiers,43219,Versailles Chantiers,...,0,0,0,SNCF,1,1,636469.5949,6.855354e+06,https://data.iledefrance-mobilites.fr/api/expl...,Versailles-Chantiers
1192,"48.88092200573709, 2.3563285436701618","{""coordinates"": [2.356328543670162, 48.8809220...",328,Gare du Nord,NaN,NaN,71410,Gare du Nord,462394,Gare du Nord,...,0,0,0,SNCF,1,1,652793.9852,6.864737e+06,https://data.iledefrance-mobilites.fr/api/expl...,Gare du Nord
1193,"48.77126603718618, 2.435306339706322","{""coordinates"": [2.435306339706322, 48.7712660...",1159,Créteil Pompadour,NaN,NaN,69950,Créteil Pompadour,46286,Créteil Pompadour,...,0,0,0,SNCF,1,0,658498.8113,6.852500e+06,https://data.iledefrance-mobilites.fr/api/expl...,Créteil Pompadour
1194,"48.8751498187932, 2.2959049047131748","{""coordinates"": [2.295904904713175, 48.8751498...",152,Charles De Gaulle-Étoile,NaN,NaN,71347,Charles-de-Gaulle Étoile,58759,Charles-de-Gaulle Étoile,...,0,0,0,RATP,1,0,648356.9845,6.864133e+06,https://data.iledefrance-mobilites.fr/api/expl...,Charles De Gaulle-Étoile


In [12]:
gares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Geo Point    1196 non-null   object 
 1   Geo Shape    1196 non-null   object 
 2   gares_id     1196 non-null   int64  
 3   nom_long     1196 non-null   object 
 4   nom_so_gar   86 non-null     object 
 5   nom_su_gar   29 non-null     object 
 6   id_ref_ZdC   1196 non-null   int64  
 7   nom_ZdC      1196 non-null   object 
 8   id_ref_ZdA   1196 non-null   int64  
 9   nom_ZdA      1196 non-null   object 
 10  idrefliga    1196 non-null   object 
 11  idrefligc    1196 non-null   object 
 12  res_com      1196 non-null   object 
 13  indice_lig   1196 non-null   object 
 14  mode_        1196 non-null   object 
 15  tertrain     1196 non-null   object 
 16  terrer       1196 non-null   object 
 17  termetro     1196 non-null   object 
 18  tertram      1196 non-null   object 
 19  terval

## Séparation Longitude - Latitude

In [13]:
gares['Latitude'], gares['Longitude'] = gares['Geo Point'].str.split(',', 1).str
gares.head()

C:\Users\Bruno\AppData\Local\Temp\ipykernel_20756\2392726816.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  gares['Latitude'], gares['Longitude'] = gares['Geo Point'].str.split(',', 1).str


,Geo Point,Geo Shape,gares_id,nom_long,nom_so_gar,nom_su_gar,id_ref_ZdC,nom_ZdC,id_ref_ZdA,nom_ZdA,...,terval,exploitant,idf,principal,x,y,picto ligne,nom_iv,Latitude,Longitude
0,"48.68101661540746, 2.533130656571037","{""coordinates"": [2.533130656571037, 48.6810166...",105,Boussy-Saint-Antoine,NaN,NaN,62596,Boussy-Saint-Antoine,47924,Boussy-Saint-Antoine,...,0,SNCF,1,0,665628.8216,6.842419e+06,https://data.iledefrance-mobilites.fr/api/expl...,Boussy-Saint-Antoine,48.68101661540746,2.533130656571037
1,"48.60674494801255, 2.3020596915826386","{""coordinates"": [2.302059691582639, 48.6067449...",108,Brétigny,NaN,NaN,60247,Brétigny,43098,Brétigny,...,0,SNCF,1,1,648544.6367,6.834288e+06,https://data.iledefrance-mobilites.fr/api/expl...,Brétigny,48.60674494801255,2.3020596915826386
2,"49.00975645120457, 2.5612326952251054","{""coordinates"": [2.561232695225105, 49.0097564...",743,Aéroport Charles De Gaulle 1,NaN,NaN,73596,Aéroport CDG 1 (Terminal 3),462398,Aéroport CDG 1 (Terminal 3) - RER,...,0,SNCF,1,0,667900.8037,6.878961e+06,https://data.iledefrance-mobilites.fr/api/expl...,Aéroport Charles De Gaulle 1,49.00975645120457,2.5612326952251054
3,"48.798892499145914, 2.3281369230647506","{""coordinates"": [2.328136923064751, 48.7988924...",22,Arcueil-Cachan,NaN,NaN,70263,Arcueil - Cachan,43067,Arcueil - Cachan,...,0,RATP,1,0,650648.8903,6.855634e+06,https://data.iledefrance-mobilites.fr/api/expl...,Arcueil-Cachan,48.798892499145914,2.3281369230647506
4,"48.932196005027805, 2.495513142584313","{""coordinates"": [2.495513142584313, 48.9321960...",956,Aulnay-sous-Bois,NaN,NaN,72646,Aulnay-sous-Bois,43071,Aulnay-sous-Bois,...,0,SNCF,1,1,663037.8693,6.870365e+06,https://data.iledefrance-mobilites.fr/api/expl...,Aulnay-sous-Bois,48.932196005027805,2.495513142584313


#### Sélection colonnes utiles

In [14]:
gares = gares[['Geo Point','Geo Shape','gares_id','nom_long','id_ref_ZdA','nom_ZdA','res_com','mode_','Latitude','Longitude']]
gares.head()

,Geo Point,Geo Shape,gares_id,nom_long,id_ref_ZdA,nom_ZdA,res_com,mode_,Latitude,Longitude
0,"48.68101661540746, 2.533130656571037","{""coordinates"": [2.533130656571037, 48.6810166...",105,Boussy-Saint-Antoine,47924,Boussy-Saint-Antoine,RER D,RER,48.68101661540746,2.533130656571037
1,"48.60674494801255, 2.3020596915826386","{""coordinates"": [2.302059691582639, 48.6067449...",108,Brétigny,43098,Brétigny,RER C,RER,48.60674494801255,2.3020596915826386
2,"49.00975645120457, 2.5612326952251054","{""coordinates"": [2.561232695225105, 49.0097564...",743,Aéroport Charles De Gaulle 1,462398,Aéroport CDG 1 (Terminal 3) - RER,RER B,RER,49.00975645120457,2.5612326952251054
3,"48.798892499145914, 2.3281369230647506","{""coordinates"": [2.328136923064751, 48.7988924...",22,Arcueil-Cachan,43067,Arcueil - Cachan,RER B,RER,48.798892499145914,2.3281369230647506
4,"48.932196005027805, 2.495513142584313","{""coordinates"": [2.495513142584313, 48.9321960...",956,Aulnay-sous-Bois,43071,Aulnay-sous-Bois,TRAIN K,TRAIN,48.932196005027805,2.495513142584313
...,...,...,...,...,...,...,...,...,...,...
1191,"48.795138178428715, 2.1351562241000215","{""coordinates"": [2.135156224100021, 48.7951381...",886,Versailles-Chantiers,43219,Versailles Chantiers,TRAIN N,TRAIN,48.795138178428715,2.1351562241000215
1192,"48.88092200573709, 2.3563285436701618","{""coordinates"": [2.356328543670162, 48.8809220...",328,Gare du Nord,462394,Gare du Nord,TRAIN H,TRAIN,48.88092200573709,2.3563285436701618
1193,"48.77126603718618, 2.435306339706322","{""coordinates"": [2.435306339706322, 48.7712660...",1159,Créteil Pompadour,46286,Créteil Pompadour,RER D,RER,48.77126603718618,2.435306339706322
1194,"48.8751498187932, 2.2959049047131748","{""coordinates"": [2.295904904713175, 48.8751498...",152,Charles De Gaulle-Étoile,58759,Charles-de-Gaulle Étoile,METRO 1,METRO,48.8751498187932,2.2959049047131748


## Enrichissement Arrondissement - Gares

In [15]:
arrondissements_gares = []

# Charger les données des gares
arrondissements_gare  = gpd.read_file('arrondissements.geojson')

# Créer une fonction pour trouver l'arrondissement d'une gare
def trouver_arrondissements(latitude, longitude):
    
    # Créer un point à partir des coordonnées de la gare
    point_gare = Point(longitude, latitude)

    # Rechercher la gare correspondant au point
    gare_serie = arrondissements_gare[arrondissements_gare.contains(point_gare)]['l_ar']
    
    if not gare_serie.empty:
        gare   = gare_serie.iloc[0]
        return gare
    else:
        return "Banlieue"

# Itérer sur chaque gare
for index, gare in gares.iterrows():
    latitude_gare  = gare['Latitude']
    longitude_gare = gare['Longitude']

    # Appeler la fonction pour trouver l'arrondissement de la gare
    arrondissement_gare = trouver_arrondissements(latitude_gare, longitude_gare)
    
    # Ajouter la gare à la liste
    arrondissements_gares.append(arrondissement_gare)

# Ajouter la liste des arrondissements comme une colonne "arrondissement" dans le DataFrame des gares
gares['arrondissement'] = arrondissements_gares

gares.head()

C:\Users\Bruno\AppData\Local\Temp\ipykernel_20756\1622097735.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gares['arrondissement'] = arrondissements_gares


,Geo Point,Geo Shape,gares_id,nom_long,id_ref_ZdA,nom_ZdA,res_com,mode_,Latitude,Longitude,arrondissement
0,"48.68101661540746, 2.533130656571037","{""coordinates"": [2.533130656571037, 48.6810166...",105,Boussy-Saint-Antoine,47924,Boussy-Saint-Antoine,RER D,RER,48.68101661540746,2.533130656571037,Banlieue
1,"48.60674494801255, 2.3020596915826386","{""coordinates"": [2.302059691582639, 48.6067449...",108,Brétigny,43098,Brétigny,RER C,RER,48.60674494801255,2.3020596915826386,Banlieue
2,"49.00975645120457, 2.5612326952251054","{""coordinates"": [2.561232695225105, 49.0097564...",743,Aéroport Charles De Gaulle 1,462398,Aéroport CDG 1 (Terminal 3) - RER,RER B,RER,49.00975645120457,2.5612326952251054,Banlieue
3,"48.798892499145914, 2.3281369230647506","{""coordinates"": [2.328136923064751, 48.7988924...",22,Arcueil-Cachan,43067,Arcueil - Cachan,RER B,RER,48.798892499145914,2.3281369230647506,Banlieue
4,"48.932196005027805, 2.495513142584313","{""coordinates"": [2.495513142584313, 48.9321960...",956,Aulnay-sous-Bois,43071,Aulnay-sous-Bois,TRAIN K,TRAIN,48.932196005027805,2.495513142584313,Banlieue


In [16]:
# Une Gare peut avoir plusieurs Sortie
gares.loc[gares['nom_long'] == 'Charles De Gaulle-Étoile'] 

,Geo Point,Geo Shape,gares_id,nom_long,id_ref_ZdA,nom_ZdA,res_com,mode_,Latitude,Longitude,arrondissement
88,"48.87420418539023, 2.2947732143825266","{""coordinates"": [2.294773214382527, 48.8742041...",151,Charles De Gaulle-Étoile,58759,Charles-de-Gaulle Étoile,RER A,RER,48.87420418539023,2.2947732143825266,17ème Ardt
871,"48.87481054018848, 2.2958012223518685","{""coordinates"": [2.295801222351868, 48.8748105...",153,Charles De Gaulle-Étoile,58759,Charles-de-Gaulle Étoile,METRO 2,METRO,48.87481054018848,2.2958012223518685,8ème Ardt
872,"48.874748649745484, 2.2955602562131467","{""coordinates"": [2.295560256213147, 48.8747486...",154,Charles De Gaulle-Étoile,58759,Charles-de-Gaulle Étoile,METRO 6,METRO,48.874748649745484,2.2955602562131467,17ème Ardt
1194,"48.8751498187932, 2.2959049047131748","{""coordinates"": [2.295904904713175, 48.8751498...",152,Charles De Gaulle-Étoile,58759,Charles-de-Gaulle Étoile,METRO 1,METRO,48.8751498187932,2.2959049047131748,17ème Ardt


## Enrichissement Communes - Gares

In [17]:
communes_gares = []

# Charger les données des communes
communes = gpd.read_file('communes-france.geojson')

# Créer une fonction pour trouver la commune d'une gare
def trouver_commune(latitude, longitude):
    
    # Créer un point à partir des coordonnées de la gare
    point = Point(longitude, latitude)

    # Rechercher la commune correspondant au point
    commune_serie = communes[communes.contains(point)]['com_name']
    
    if not commune_serie.empty:
        commune   = commune_serie.iloc[0]
        return commune
    else:
        return ""

# Itérer sur chaque gare
for index, gare in gares.iterrows():
    latitude_gare  = gare['Latitude']
    longitude_gare = gare['Longitude']

    # Appeler la fonction pour trouver la commune de la gare
    commune_gare   = trouver_commune(latitude_gare, longitude_gare)
    
    # Ajouter la commune à la liste
    communes_gares.append(commune_gare)

    # Ajouter la liste des communes comme une colonne "commune" dans le DataFrame des gares
gares['commune']   = communes_gares

gares.head()

C:\Users\Bruno\AppData\Local\Temp\ipykernel_20756\3024481489.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gares['commune'] = communes_gares


,Geo Point,Geo Shape,gares_id,nom_long,id_ref_ZdA,nom_ZdA,res_com,mode_,Latitude,Longitude,arrondissement,commune
0,"48.68101661540746, 2.533130656571037","{""coordinates"": [2.533130656571037, 48.6810166...",105,Boussy-Saint-Antoine,47924,Boussy-Saint-Antoine,RER D,RER,48.68101661540746,2.533130656571037,Banlieue,Boussy-Saint-Antoine
1,"48.60674494801255, 2.3020596915826386","{""coordinates"": [2.302059691582639, 48.6067449...",108,Brétigny,43098,Brétigny,RER C,RER,48.60674494801255,2.3020596915826386,Banlieue,Brétigny-sur-Orge
2,"49.00975645120457, 2.5612326952251054","{""coordinates"": [2.561232695225105, 49.0097564...",743,Aéroport Charles De Gaulle 1,462398,Aéroport CDG 1 (Terminal 3) - RER,RER B,RER,49.00975645120457,2.5612326952251054,Banlieue,Tremblay-en-France
3,"48.798892499145914, 2.3281369230647506","{""coordinates"": [2.328136923064751, 48.7988924...",22,Arcueil-Cachan,43067,Arcueil - Cachan,RER B,RER,48.798892499145914,2.3281369230647506,Banlieue,Cachan
4,"48.932196005027805, 2.495513142584313","{""coordinates"": [2.495513142584313, 48.9321960...",956,Aulnay-sous-Bois,43071,Aulnay-sous-Bois,TRAIN K,TRAIN,48.932196005027805,2.495513142584313,Banlieue,Aulnay-sous-Bois


In [18]:
gares.shape

(1196, 12)

#### Filtre pour enlever les TRAMWAY et VAL

In [19]:
gares = gares[~gares['mode_'].isin(['TRAMWAY', 'VAL'])]
gares.shape

(930, 12)

In [20]:
print(gares['res_com'].unique())

['RER D' 'RER C' 'RER B' 'TRAIN K' 'TRAIN H' 'TRAIN L' 'RER A' 'TRAIN J'
 'TRAIN R' 'TRAIN P' 'RER E' 'TRAIN N' 'TRAIN U' 'GL' 'METRO 10' 'METRO 4'
 'METRO 3' 'METRO 9' 'METRO 2' 'METRO 7bis' 'METRO 5' 'METRO 1' 'METRO 14'
 'METRO 8' 'METRO 6' 'METRO 11' 'METRO 13' 'METRO 12' 'METRO 7'
 'METRO 3bis']


## Exportation Fichier Gare to CSV

In [21]:
gares.to_csv("liste_gares_cleaned.csv", index = False)

# -----------------------------

# ENRICHISSEMENT LIGNES GRAND PARIS

In [22]:
grand_paris = pd.read_csv("point-de-localisation-des-gares-du-grand-paris-express_2.csv", sep = ';')
grand_paris['Latitude'], grand_paris['Longitude'] = grand_paris['Geo Point'].str.split(',', 1).str
grand_paris.head()

C:\Users\Bruno\AppData\Local\Temp\ipykernel_20756\3453295926.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  grand_paris['Latitude'], grand_paris['Longitude'] = grand_paris['Geo Point'].str.split(',', 1).str


,Geo Point,Geo Shape,OBJECTID,LIBELLE,CONNEX,LIGNE,Latitude,Longitude,arrondissement,commune
0,"48.89085479122746, 2.237688709843251","{""coordinates"": [2.237688709843251, 48.8908547...",77,La Défense,Interconnexion,L15,48.89085479122746,2.237688709843251,Banlieue,Puteaux
1,"48.914987385627605, 2.2720135095825587","{""coordinates"": [2.272013509582559, 48.9149873...",79,Bois-Colombes,Interconnexion,L15,48.914987385627605,2.2720135095825587,Banlieue,Bois-Colombes
2,"48.729934252451855, 2.1613837430608083","{""coordinates"": [2.161383743060808, 48.7299342...",82,CEA Saint-Aubin,Non,L18,48.729934252451855,2.1613837430608083,Banlieue,Saclay
3,"48.71536419480408, 2.2009585832079868","{""coordinates"": [2.200958583207987, 48.7153641...",83,Palaiseau,Non,L18,48.71536419480408,2.2009585832079868,Banlieue,Palaiseau
4,"48.945530309114524, 2.4627419442570635","{""coordinates"": [2.462741944257063, 48.9455303...",87,Le Blanc-Mesnil,Non,L16,48.945530309114524,2.4627419442570635,Banlieue,Le Blanc-Mesnil
5,"48.93058045704873, 2.422115109237471","{""coordinates"": [2.422115109237471, 48.9305804...",88,Le Bourget RER,Interconnexion,L16/L17,48.93058045704873,2.422115109237471,Banlieue,Le Bourget
6,"48.72514427813822, 2.2584664922454603","{""coordinates"": [2.25846649224546, 48.72514427...",89,Massy - Palaiseau,Interconnexion,L18,48.72514427813822,2.2584664922454603,Banlieue,Massy
7,"48.71215046689281, 2.173294535262749","{""coordinates"": [2.173294535262749, 48.7121504...",90,Orsay - Gif,Non,L18,48.71215046689281,2.173294535262749,Banlieue,Orsay
8,"48.82142330304046, 2.3590830775080582","{""coordinates"": [2.359083077508058, 48.8214233...",91,Maison Blanche - Paris XIIIe,Interconnexion,L14,48.82142330304046,2.3590830775080582,13ème Ardt,Paris
9,"48.91768277512885, 2.346560389198723","{""coordinates"": [2.346560389198723, 48.9176827...",92,Saint-Denis Pleyel,Interconnexion,L14/L15/L16/L17,48.91768277512885,2.346560389198723,Banlieue,Saint-Denis


In [24]:
# Sélection des colonnes utiles
grand_paris = grand_paris[['Geo Point','Geo Shape','OBJECTID','LIBELLE','CONNEX','LIGNE','Latitude','Longitude']]
grand_paris.head()

,Geo Point,Geo Shape,OBJECTID,LIBELLE,CONNEX,LIGNE,Latitude,Longitude
0,"48.89085479122746, 2.237688709843251","{""coordinates"": [2.237688709843251, 48.8908547...",77,La Défense,Interconnexion,L15,48.89085479122746,2.237688709843251
1,"48.914987385627605, 2.2720135095825587","{""coordinates"": [2.272013509582559, 48.9149873...",79,Bois-Colombes,Interconnexion,L15,48.914987385627605,2.2720135095825587
2,"48.729934252451855, 2.1613837430608083","{""coordinates"": [2.161383743060808, 48.7299342...",82,CEA Saint-Aubin,Non,L18,48.729934252451855,2.1613837430608083
3,"48.71536419480408, 2.2009585832079868","{""coordinates"": [2.200958583207987, 48.7153641...",83,Palaiseau,Non,L18,48.71536419480408,2.2009585832079868
4,"48.945530309114524, 2.4627419442570635","{""coordinates"": [2.462741944257063, 48.9455303...",87,Le Blanc-Mesnil,Non,L16,48.945530309114524,2.4627419442570635


### Enrichissement Arrondissement - Grand Paris

In [25]:
arrondissements_grand_paris = []

# Charger les données des arrondissements
arrondissements = gpd.read_file('arrondissements.geojson')

# Créer une fonction pour trouver l'arrondissement d'un arrêt du Grand Paris
def trouver_arrondissements(latitude, longitude):
    
    # Créer un point à partir des coordonnées de l'arrêt du Grand Paris
    point_grand_paris = Point(longitude, latitude)

    # Rechercher l'arrondissement correspondant au point
    grand_paris_serie = arrondissements[arrondissements.contains(point_grand_paris)]['l_ar']
    
    if not grand_paris_serie.empty:
        grand_paris   = grand_paris_serie.iloc[0]
        return grand_paris
    else:
        return "Banlieue"

# Itérer sur chaque arrêt du Grand Paris
for index, arret in grand_paris.iterrows():
    latitude_arret  = arret['Latitude']
    longitude_arret = arret['Longitude']

    # Appeler la fonction pour trouver l'arrondissement de l'arrêt du Grand Paris
    arrondissement_grand_paris = trouver_arrondissements(latitude_arret, longitude_arret)
    
    # Ajouter l'arrêt à la liste
    arrondissements_grand_paris.append(arrondissement_grand_paris)

# Ajouter la liste des arrondissements comme une colonne "arrondissement" dans le DataFrame des arrêts du Grand Paris
grand_paris['arrondissement']  = arrondissements_grand_paris

grand_paris.head()

,Geo Point,Geo Shape,OBJECTID,LIBELLE,CONNEX,LIGNE,Latitude,Longitude,arrondissement
0,"48.89085479122746, 2.237688709843251","{""coordinates"": [2.237688709843251, 48.8908547...",77,La Défense,Interconnexion,L15,48.89085479122746,2.237688709843251,Banlieue
1,"48.914987385627605, 2.2720135095825587","{""coordinates"": [2.272013509582559, 48.9149873...",79,Bois-Colombes,Interconnexion,L15,48.914987385627605,2.2720135095825587,Banlieue
2,"48.729934252451855, 2.1613837430608083","{""coordinates"": [2.161383743060808, 48.7299342...",82,CEA Saint-Aubin,Non,L18,48.729934252451855,2.1613837430608083,Banlieue
3,"48.71536419480408, 2.2009585832079868","{""coordinates"": [2.200958583207987, 48.7153641...",83,Palaiseau,Non,L18,48.71536419480408,2.2009585832079868,Banlieue
4,"48.945530309114524, 2.4627419442570635","{""coordinates"": [2.462741944257063, 48.9455303...",87,Le Blanc-Mesnil,Non,L16,48.945530309114524,2.4627419442570635,Banlieue


### Enrichissement Communes - Grand Paris

In [26]:
communes_grand_paris = []

# Charger les données des communes
communes = gpd.read_file('communes-france.geojson')

# Créer une fonction pour trouver la commune d'un arrêt du Grand Paris
def trouver_communes(latitude, longitude):
    
    # Créer un point à partir des coordonnées de la commune du Grand Paris
    point_grand_paris = Point(longitude, latitude)

    # Rechercher la commune correspondant au point
    commune_gp_serie  = communes[communes.contains(point_grand_paris)]['com_name']
    
    if not commune_gp_serie.empty:
        commune_gp    = commune_gp_serie.iloc[0]
        return commune_gp
    else:
        return ""

# Itérer sur chaque arrêt du Grand Paris
for index, arret in grand_paris.iterrows():
    latitude_arret  = arret['Latitude']
    longitude_arret = arret['Longitude']

    # Appeler la fonction pour trouver la commune de l'arrêt du Grand Paris
    commune_grand_paris = trouver_communes(latitude_arret, longitude_arret)
    
    # Ajouter l'arrêt à la liste
    communes_grand_paris.append(commune_grand_paris)

# Ajouter la liste des communes comme une colonne "commune" dans le DataFrame des arrêts du Grand Paris
grand_paris['commune']  = communes_grand_paris

grand_paris.head()

,Geo Point,Geo Shape,OBJECTID,LIBELLE,CONNEX,LIGNE,Latitude,Longitude,arrondissement,commune
0,"48.89085479122746, 2.237688709843251","{""coordinates"": [2.237688709843251, 48.8908547...",77,La Défense,Interconnexion,L15,48.89085479122746,2.237688709843251,Banlieue,Puteaux
1,"48.914987385627605, 2.2720135095825587","{""coordinates"": [2.272013509582559, 48.9149873...",79,Bois-Colombes,Interconnexion,L15,48.914987385627605,2.2720135095825587,Banlieue,Bois-Colombes
2,"48.729934252451855, 2.1613837430608083","{""coordinates"": [2.161383743060808, 48.7299342...",82,CEA Saint-Aubin,Non,L18,48.729934252451855,2.1613837430608083,Banlieue,Saclay
3,"48.71536419480408, 2.2009585832079868","{""coordinates"": [2.200958583207987, 48.7153641...",83,Palaiseau,Non,L18,48.71536419480408,2.2009585832079868,Banlieue,Palaiseau
4,"48.945530309114524, 2.4627419442570635","{""coordinates"": [2.462741944257063, 48.9455303...",87,Le Blanc-Mesnil,Non,L16,48.945530309114524,2.4627419442570635,Banlieue,Le Blanc-Mesnil


## Exportation Fichier Grand Paris to CSV

In [27]:
grand_paris.to_csv("point-de-localisation-des-gares-du-grand-paris-express.csv", sep=";",index=False)

## Création de la fonction calculant le nombre de stations autour d'une gare

In [29]:
grand_paris['Latitude'], grand_paris['Longitude'] = grand_paris['Geo Point'].str.split(',', 1).str


def compter_stations_dans_rayon(lat_position, lon_position, stations, rayon):
    nb_stations = 0
    position_coords = (lat_position, lon_position)

    for index, row in stations.iterrows():
        if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
            coords_station = (row['Latitude'], row['Longitude'])
            distance = geopy.distance.distance(position_coords, coords_station).m
            if distance <= rayon:
                nb_stations += 1

    return nb_stations

resultats_gare = []

# Compter le nombre de stations Vélib dans différents rayons autour de chaque position de gare
rayons = [100, 200, 300,400]  # Liste des rayons en mètres

for index, row in gares.iterrows():
    lat_gare = row['Latitude']
    lon_gare = row['Longitude']

    for rayon in rayons:
        nb_stations = compter_stations_dans_rayon(lat_gare, lon_gare, stations, rayon)
        resultats_gare.append({'Gare': row['nom_long'], 'Coordonnées': row['Geo Point'], 'Ligne': row['res_com'], f'Nombre de stations ({rayon}m)': nb_stations})

# Créer un DataFrame à partir de la liste resultats_gare
df_resultats = pd.DataFrame(resultats_gare)

# Exporter le DataFrame vers un fichier CSV
df_resultats.to_csv('rayonnage_gare_gpe.csv', index=False)

print("Le fichier CSV a été généré avec succès.")


C:\Users\Bruno\AppData\Local\Temp\ipykernel_20756\3624806526.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  grand_paris['Latitude'], grand_paris['Longitude'] = grand_paris['Geo Point'].str.split(',', 1).str


Le fichier CSV a été généré avec succès.
